In [9]:
import pandas as pd
import os
import cv2
import numpy as np
import json

import imutils
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

#Load index list
Year='1935'
Showa='10'


In [3]:
#Encoding Function
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [4]:
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Image):
    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    

In [26]:
#Function for Cell
def GetCell(cropped):
    #Code for Adding Grid
        ##Right page
        img = cropped.copy()
        hh, ww = img.shape[:2]

        #Identify grid location
        ## convert to grayscale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # threshold gray image
        thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

        ## count number of non-zero pixels in each column and row. 
        countCol = np.count_nonzero(thresh, axis=0)
        countRow = np.count_nonzero(thresh, axis=1)

        ###############
        ## Column lines
        ###############
        ### This finds the height of the smallest peak
        peaksCol, _ = find_peaks(countCol, distance=10)
        ### threshold count at Thres
        count_thresh = countCol.copy()
        count_thresh[peaksCol] = 255
        count_thresh[count_thresh!=255] = 0
        count_thresh = count_thresh.astype(np.uint8)

        ### get contours
        contoursCol = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contoursCol = contoursCol[0] if len(contoursCol) == 2 else contoursCol[1]

        ### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
        result = cropped.copy()
        for cntr in contoursCol:
            x,y,w,h = cv2.boundingRect(cntr)
            ycenter = y
            cv2.line(result, (ycenter,0), (ycenter,hh), (255, 0, 0), 1)
        

        ###############
        ## Row lines
        ###############
        peaksRow, _ = find_peaks(countRow, distance=60)
        ### threshold count at Thres
        count_thresh = countRow.copy()
        count_thresh[peaksRow] = 255
        count_thresh[count_thresh!=255] = 0
        count_thresh = count_thresh.astype(np.uint8)

        ### get contours
        contoursRow = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contoursRow = contoursRow[0] if len(contoursRow) == 2 else contoursRow[1]

        ### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
        for cntr in contoursRow:
            x,y,w,h = cv2.boundingRect(cntr)
            ycenter = y+h//2
            cv2.line(result, (0,ycenter), (hh,ycenter), (255, 0, 0), 1)
                
        cv2.imshow("Result",result)
        cv2.waitKey(0)
        return(peaksRow,peaksCol)

In [6]:
def Extract(Position,ImageNumber):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
    stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
    bytes = bytearray(stream.read())
    numpyarray = np.asarray(bytes, dtype=np.uint8)
    img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)

    HH,WW=img.shape[:2]
    
    df = pd.DataFrame(columns = ['Name', 'Wage'])
    if Position=="Admin":
        cropped=img[0:HH//2,0:WW]
        MiddleLine=GetCell(cropped)[0]
        ColumnLine=GetCell(cropped)[1]
        
        ##Top Row ##
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        HH,WW=cropped.shape[:2]
        cv2.imshow("Sample",cropped)
        cv2.waitKey(0)
        for Line in ColumnLine.tolist():
            if Line==ColumnLine.tolist()[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                df = df.append(df2, ignore_index = True)
            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfT = df.append(df2, ignore_index = True)

        ##Bottom Row##
        cropped=img[HH//2:HH,0:WW]
        HH,WW=cropped.shape[:2]
        MiddleLine=GetCell(cropped)[0]
        ColumnLine=GetCell(cropped)[1]
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        cv2.imshow("Sample",cropped)
        cv2.waitKey(0)
        for Line in ColumnLine.tolist():
            if Line==ColumnLine.tolist()[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                df = df.append(df2, ignore_index = True)
            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                df = df.append(df2, ignore_index = True)
                
                
                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfB = df.append(df2, ignore_index = True)        
        return dfT.append(dfB)

In [10]:
#Load Data Frame
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\"+str(Year)+"\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+str(Showa)+'.csv')
df=df.drop(df.columns[0], axis=1)

In [15]:
#Test code| Version 2#
#Show Working office#
n=1

#Extract key info of office
Row  = df.iloc[n]
NxRow= df.iloc[n+1]
Row, NxRow

###Collect Location information###
Page=int(Row["Page"])
NxPage=int(NxRow["Page"])

Dept=Row["Dept"]
NxDept=NxRow["Dept"]

Office=Row["Office"]
NxOffice=NxRow["Office"]

PositionList=list(dta[str(Year)][Dept][Office]['Position'].keys())
PositionList

#Convert to json via CLOVA
Position=PositionList[2]
StrPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['Page'])
EndPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['EndPage'])
PageList=list(set([1,EndPage-StrPage+1]))

ImageNumber=PageList[0]
ImageNumber,Dept,Office,Position,

(1, 'Admin', '職員課', 'Admin')

In [16]:
#Download Image
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
bytes = bytearray(stream.read())
numpyarray = np.asarray(bytes, dtype=np.uint8)
img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
HH,WW=img.shape[:2]

In [17]:
#For Multi-row image
#Top Row
#Creart data storage file
df=pd.DataFrame(columns = ['Name', 'Wage'])

#Crop 
cropped=img[0:HH//2,0:WW]
cv2.imshow("Sample",cropped)
cv2.waitKey(0)
HHH=cropped.shape[:2][0]
MiddleLineList=GetCell(cropped)[0]
res = list(map(abs, [d-HHH//2 for d in MiddleLineList.tolist()]))
minpos = res.index(min(res))
MiddleLine=MiddleLineList[minpos]

ColumnLine=GetCell(cropped)[1]
MiddleLine=MiddleLineList[minpos]

path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
for Line in ColumnLine.tolist():
    if Line==ColumnLine.tolist()[0]:
        #Wage
        Image=cropped[0:MiddleLine,0:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Wage='NA'
        else:
            Wage=''.join([d['inferText'] for d in Json['fields']])

        #Name
        Image=cropped[MiddleLine:HH,0:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Name='NA'
        else:
            Name=''.join([d['inferText'] for d in Json['fields']])

        #Add to DF
        df2 = {'Name': Name, 'Wage': Wage}
        df = df.append(df2, ignore_index = True)
    else:
        #Wage
        Image=cropped[0:MiddleLine,Line-15:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Wage='NA'
        else:
            Wage=''.join([d['inferText'] for d in Json['fields']])

        #Name
        Image=cropped[MiddleLine:HH,Line-15:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Name='NA'
        else:
            Name=''.join([d['inferText'] for d in Json['fields']])

        #Add to DF
        df2 = {'Name': Name, 'Wage': Wage}
        df = df.append(df2, ignore_index = True)
dfT=df

C:\Temp\ipykernel_29772\815608455.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\815608455.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

In [27]:
#Bottom Row
#Creart data storage file
df=pd.DataFrame(columns = ['Name', 'Wage'])

#Crop
cropped=img[HH//2:HH,0:WW]
cv2.imshow("Sample",cropped)
cv2.waitKey(0)
HHH=cropped.shape[:2][0]
MiddleLineList=GetCell(cropped)[0]
res = list(map(abs, [d-HHH//2 for d in MiddleLineList.tolist()]))
minpos = res.index(min(res))
MiddleLine=MiddleLineList[minpos]

ColumnLine=GetCell(cropped)[1]
MiddleLine=MiddleLineList[minpos]



In [28]:
MiddleLineList

array([ 29, 148], dtype=int64)

In [22]:
MiddleLine=MiddleLineList[0]
ColumnLine=GetCell(cropped)[1]

path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
cv2.imshow("Sample",cropped)
cv2.waitKey(0)
for Line in ColumnLine.tolist():
    if Line==ColumnLine.tolist()[0]:
        #Wage
        Image=cropped[0:MiddleLine,0:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Wage='NA'
        else:
            Wage=''.join([d['inferText'] for d in Json['fields']])

        #Name
        Image=cropped[MiddleLine:HHH,0:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Name='NA'
        else:
            Name=''.join([d['inferText'] for d in Json['fields']])

        #Add to DF
        df2 = {'Name': Name, 'Wage': Wage}
        df = df.append(df2, ignore_index = True)

    else:
        #Wage
        Image=cropped[0:MiddleLine,Line-15:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Wage='NA'
        else:
            Wage=''.join([d['inferText'] for d in Json['fields']])

        #Name
        Image=cropped[MiddleLine:HHH,Line-15:Line]
        cv2.imwrite(path+"Temp.jpg",Image)
        with open(path+"Temp.jpg",'rb') as f:
            file_data = f.read()
        Json=Clova(file_data)
        if Json['inferResult']=='ERROR':
            Name='NA'
        else:
            Name=''.join([d['inferText'] for d in Json['fields']])

        #Add to DF
        df2 = {'Name': Name, 'Wage': Wage}
        df = df.append(df2, ignore_index = True)
dfB=df

C:\Temp\ipykernel_29772\1628962281.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\1628962281.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\1628962281.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\1628962281.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
C:\Temp\ipykernel_29772\1628962281.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

In [30]:
DF=pd.concat([dfT,dfB])
DF=DF[(DF['Name']!="")]
DF=DF[(DF['Name']!="NA")]
DF=DF[(DF['Wage']!="")]
DF=DF[(DF['Wage']!="NA")]

DF

,Name,Wage
5,白蟋卵,上老村
6,野博,月T
7,木下浩三郎,月七日
8,立花昌夫,1
9,河野秀夫,1子
10,佐蒜重,不
11,藤本修平,月九五
12,佐###靖,:
13,高石源治,下
5,白蟋卵,上老村


In [24]:
dta[Year][Dept][Office]['Position'][Position]['Data']=DF.to_dict('records')
dta[Year][Dept][Office]['Position'][Position]

{'XLocation': 242,
 'Page': 3,
 'EndLocation': 513.0,
 'EndPage': 4.0,
 'Data': [{'Name': '白蟋卵', 'Wage': '上老村'},
  {'Name': '野博', 'Wage': '月T'},
  {'Name': '木下浩三郎', 'Wage': '月七日'},
  {'Name': '立花昌夫', 'Wage': '1'},
  {'Name': '河野秀夫', 'Wage': '1子'},
  {'Name': '佐蒜重', 'Wage': '不'},
  {'Name': '藤本修平', 'Wage': '月九五'},
  {'Name': '佐###靖', 'Wage': ':'},
  {'Name': '高石源治', 'Wage': '下'},
  {'Name': '1', 'Wage': ''},
  {'Name': '白蟋卵', 'Wage': '上老村'},
  {'Name': '野博', 'Wage': '月T'},
  {'Name': '木下浩三郎', 'Wage': '月七日'},
  {'Name': '立花昌夫', 'Wage': '1'},
  {'Name': '河野秀夫', 'Wage': '1子'},
  {'Name': '佐蒜重', 'Wage': '不'},
  {'Name': '藤本修平', 'Wage': '月九五'},
  {'Name': '佐###靖', 'Wage': ':'},
  {'Name': '高石源治', 'Wage': '下'},
  {'Name': '1', 'Wage': ''},
  {'Name': '白蟋卵', 'Wage': '上老村'},
  {'Name': '野博', 'Wage': '月T'},
  {'Name': '木下浩三郎', 'Wage': '月七日'},
  {'Name': '立花昌夫', 'Wage': '1'},
  {'Name': '河野秀夫', 'Wage': '1子'},
  {'Name': '佐蒜重', 'Wage': '不'},
  {'Name': '藤本修平', 'Wage': '月九五'},
  {'Name': '佐###靖', '

In [204]:
Extract(Position,ImageNumber)

TypeError: 'NoneType' object is not iterable

In [70]:
###Collect Location information###
Page=int(Row["Page"])
NxPage=int(NxRow["Page"])

Dept=Row["Dept"]
NxDept=NxRow["Dept"]

Office=Row["Office"]
NxOffice=NxRow["Office"]

PositionList=list(dta[str(Year)][Dept][Office]['Position'].keys())
PositionList

#Convert to json via CLOVA
Position=PositionList[2]
StrPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['Page'])
EndPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['EndPage'])
PageList=list(set([1,EndPage-StrPage+1]))

Extract(Position,ImageNumberPageList[0])

[1]

In [82]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)